## Email Analysis *Extract, Load, and Transform* Notebook

### Imports

In [1]:
# Standard Library Imports
import logging
from typing import List, Dict, Any
from datetime import datetime

# Third-Party Imports
import pypff # type: ignore

# Local Imports
from src.utils.config import Config
from src.extract.pst_message_extractor import PstMessageExtractor, PstMessage
from src.extract.message_parser import MessageParser, ParsedMessage
from src.transform.message_enricher import MessageEnricher, EnrichedMessage
#from src.load.data_loader import DataLoader, DataLoadResult, DataLoadError

logging.basicConfig(level=logging.INFO)

### Configuration

In [2]:
config: Config = Config.from_json("config.json")
extractor: PstMessageExtractor = PstMessageExtractor(config.input_pst_path, config.chunk_size)
message_parser: MessageParser = MessageParser()
#derived_extractor: DerivedFeaturesExtractor = DerivedFeaturesExtractor()
#loader: DataLoader = DataLoader(config.output_directory)

### ETL Pipeline with Error Handling

In [4]:
for message_batch in extractor.extract_messages():
        #enriched_messages: List[EnrichedMessage] = []
        #failed_messages: List[Union[PstExtractError, ParseError, EnrichError, 
        
        for pst_message in message_batch.messages:
            parsed_message: ParsedMessage = message_parser.extract(pst_message.message, pst_message.folder_name)
            #enriched_message: EnrichedMessage = MessageEnricher.enrich(parsed_message)
            
            #enriched_messages.append(enriched_message)
        
        #loader.load(enriched_messages)

INFO:root:Skipping empty folder: Deleted Items
INFO:root:Extracting messages from folder: Inbox
ERROR:root:Error getting value for key conversation_topic: pypff_message_get_conversation_topic: unable to retrieve conversation topic size. libuna_unicode_character_copy_from_utf16_stream: unsupported UTF-16 character. libuna_utf8_string_size_from_utf16_stream: unable to copy Unicode character from UTF-16 stream. libpff_mapi_value_get_data_as_utf8_string_size: unable to determine size of value data as UTF-8 string. libpff_record_entry_get_data_as_utf8_string_size_with_codepage: unable to determine size of value data as UTF-8 string. libpff_internal_item_get_entry_value_utf8_string_size: unable to retrieve UTF-8 string size. libpff_message_get_entry_value_utf8_string_size: unable to retrieve UTF-8 string size.
INFO:root:Skipping empty folder: Outbox
INFO:root:Skipping empty folder: Sync Issues (This computer only)
INFO:root:Skipping empty folder: [Gmail]
INFO:root:Skipping empty folder: Bank